# Credit Risk Resampling Techniques

In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [24]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [25]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)


#df[["issue_d","next_pymnt_d"]] = pd.to_datetime(df[["issue_d","next_pymnt_d"]]).strftime('%m-%Y')
#df[["issue_d","next_pymnt_d"]] = pd.to_datetime(df[["issue_d","next_pymnt_d"]]).view(int)//10**9

#df = df.drop(columns="issue_d")
#df = df.drop(columns="next_pymnt_d")

# column_names = df.select_dtypes(include="object")
# print(column_names)
# for column_name in column_names:
#     df[column_name] = df[column_name].astype('string')
# #df.to_csv("check_data.csv")
# column_names = df.select_dtypes(include="object")
# print(column_names)
# df.head()

In [26]:
#df[["home_ownership", "verification_status"]] = df[["home_ownership", "verification_status"]].astype('category')

#df["home_ownership"]
#df["verification_status"]
#df["home_ownership"].replace(['ANY','MORTGAGE','OWN','RENT'],[0,1,2,3], inplace=True)
#df["verification_status"].replace(['Not Verified','Source Verified','Verified'],[0,1,2], inplace=True)

#df[["pymnt_plan", "hardship_flag", "debt_settlement_flag","initial_list_status","application_type"]] = df[["pymnt_plan", "hardship_flag", "debt_settlement_flag","initial_list_status","application_type"]].astype('category')
#df["pymnt_plan"].replace(['n'],[0], inplace=True)
#df["hardship_flag"].replace(['N'],[0], inplace=True)
#df["debt_settlement_flag"].replace(['N'],[0], inplace=True)
#df["initial_list_status"].replace(['f','w'],[0,1],inplace=True)
#df["application_type"].replace(['Individual','Joint App'],[0,1], inplace=True)


# Split the Data into Training and Testing

In [27]:
# Create our target
y = df["loan_status"]

# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)


In [28]:
X.describe()
print(type(X))
print(X.select_dtypes(include=['category']))

<class 'pandas.core.frame.DataFrame'>
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[68817 rows x 0 columns]


In [29]:
# Check the balance of our target values
y.head()
print(type(y))
y.value_counts()

<class 'pandas.core.series.Series'>


low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)
X_train.shape

(51612, 95)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [31]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [32]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)



array([[   72,    29],
       [ 6582, 10522]])

In [34]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.6640245116653853

In [35]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.62      0.02      0.66      0.44       101
   low_risk       1.00      0.62      0.71      0.76      0.66      0.43     17104

avg / total       0.99      0.62      0.71      0.76      0.66      0.43     17205



### SMOTE Oversampling

In [36]:
# Resample the training data with SMOTE
# YOUR CODE HERE

from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
#X_resampled.dropna(axis='columns', how='all', inplace=True)
#X_resampled.dropna(inplace=True)
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [37]:
#column_names = X_resampled.select_dtypes(include='category')
#column_names
#X_resampled[['home_ownership','verification_status','initial_list_status']] = pd.get_dummies(data=X_resampled, columns=['home_ownership', 'verification_status','initial_list_status'])
#X_resampled

In [38]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [39]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

y_pred_SMOTE = model.predict(X_test)
confusion_matrix(y_test, y_pred_SMOTE)


array([[   64,    37],
       [ 5514, 11590]])

In [40]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_SMOTE)

0.6556413183413758

In [41]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_SMOTE))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.68      0.02      0.66      0.43       101
   low_risk       1.00      0.68      0.63      0.81      0.66      0.43     17104

avg / total       0.99      0.68      0.63      0.80      0.66      0.43     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [46]:
X_train.dtypes
X_train.head()
column_names = X_train.select_dtypes(include='uint8')
column_names
for column_name in column_names:
    X_train[column_name] = X_train[column_name].astype('float64')

X_train.dtypes


loan_amnt                      float64
int_rate                       float64
installment                    float64
annual_inc                     float64
dti                            float64
                                ...   
next_pymnt_d_May-2019          float64
application_type_Individual    float64
application_type_Joint App     float64
hardship_flag_N                float64
debt_settlement_flag_N         float64
Length: 95, dtype: object

In [47]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)



Counter({'high_risk': 246, 'low_risk': 246})

In [48]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [49]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

y_pred_us = model.predict(X_test)
confusion_matrix(y_test, y_pred_us)

array([[   68,    33],
       [10429,  6675]])

In [50]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_us)

0.5317634575665238

In [51]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_us))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.39      0.01      0.51      0.27       101
   low_risk       1.00      0.39      0.67      0.56      0.51      0.26     17104

avg / total       0.99      0.39      0.67      0.56      0.51      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [52]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68460, 'low_risk': 62011})

In [53]:
# Train the Logistic Regression model using the resampled data
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [54]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix

y_pred_SMOTEEN = model.predict(X_test)
confusion_matrix(y_test, y_pred_SMOTEEN)

array([[  73,   28],
       [7309, 9795]])

In [55]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_SMOTEEN)

0.6477226680806527

In [56]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_SMOTEEN))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.57      0.02      0.64      0.42       101
   low_risk       1.00      0.57      0.72      0.73      0.64      0.41     17104

avg / total       0.99      0.57      0.72      0.72      0.64      0.41     17205

